# Potential interesting Questions:
- What are the games with either very good or very bad rating (high stddev)? Those might be the interesting ones. sort out more.

- What does influence the rating?
    - Does game length ~?
    - Do prizes (honors column) ~?

### rows dropped
- if accessory rank is int, because I'm not interested in accessories

### columns dropped  
- version
- compilation
- accessory
- implementation
- expansion (replaced by expansion count)
- thumbnail
- commerce
- honor (replaced by honor count)


[Kaggle link](https://www.kaggle.com/datasets/seanthemalloy/board-game-geek-database/)

[API description](https://boardgamegeek.com/wiki/page/BGG_XML_API2#toc3)

## ToDo
- Features reduzieren siehe aggregate_mlb_clean
- Normalisierung der Linear Regression hinkriegen


### Notes
- too many publisher, family: Narrow down to the biggest 10 + stuff like kickstarter, web & selfpublished
- sub-domain ok for one-hot encoding
- category, mechanism  vielleicht zuviel für 1-hot
- way to many artists and designer, biggest category uncredited -> reduce drastically

In [ ]:
import numpy as np
import pandas as pd
import os
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
pd.set_option("display.max_rows", 500)

In [ ]:
data_folder = "data"
file = os.path.join(data_folder, "BGGdata.csv")

In [ ]:
if os.path.isfile(file):
    bgg_df = pd.read_csv(file)
else:
    os.system("unzip " + file + ".zip -d " + data_folder)
    bgg_df = pd.read_csv(file)

In [ ]:
bgg_df2 = bgg_df.copy()

# Parameters 

In [ ]:
userrated_cutoff = 10 # line with less ratings get deleted

# Preprocessing
## deleting rows containing accessories, not games as well as row without too few ratings

In [ ]:
bgg_df2 = bgg_df2[bgg_df2["accessoryrank"].isna()]

In [ ]:
bgg_df2 = bgg_df2[bgg_df2["usersrated"] >= userrated_cutoff]
#bgg_df2.sort_values(["usersrated"], ascending=False).filter(like="name").head(30)


## converting some categories to counts

In [ ]:
# Honor count is more interesting than which honors
bgg_df2["hon_count"] = bgg_df2.filter(like="honor").count(axis=1)
# expansion count is more interesting than which expansions
bgg_df2["expan_count"] = bgg_df.filter(like="expansion").count(axis=1)


## dropping uninteresting columns 

In [ ]:
def drop_columns(df:pd.DataFrame, search_string:list = None)-> None:
    '''
    Without search_string function prompts for 
    With search_string deleted columns whose title includes the search_string
    '''
    if search_string:
        search_string =  search_string if type(search_string) == str else "(?i)"+"|".join(search_string)
        df_temp = df.filter(regex=search_string)
        drop_column = df_temp.columns
        df.drop(columns=drop_column, inplace = True)
        return None

    search_string = input("filter string:")
    df_temp = df.filter(like=search_string)
    drop_column = df_temp.columns #if regex else df.filter(like=search_string).columns
    display(drop_column)
    
    answer = input("drop? yes/display/exit/")

    if answer == "yes":
        df.drop(columns=drop_column, inplace = True)
        drop_columns(df)
    elif answer == "display":
        display(df[drop_column].dropna().head())
        drop_columns(df)
    elif answer == "exit":
        return None
    else:
        drop_columns(df)

In [ ]:
# dropping uninteresting columns
drop_columns(bgg_df2, ["version","compilation","accessory", "implementation", "expansion", "thumbnail","commerce","honor"])

# categorical variables
## exploring

In [ ]:
def sum_value_counts(df:pd.DataFrame, columns:str):
    uniques = set()
    temp_df = pd.DataFrame()
    for column in df.filter(like=columns):
        uniques = uniques.union(set(df[column].unique()))
        temp_df[column] = df[column].value_counts()
    df_out = temp_df.sum(axis=1).sort_values(ascending=False)
    return df_out, uniques


In [ ]:
honor_counts = sum_value_counts(bgg_df.drop(columns='familyrank'), "category")[0]
honor_counts.sort_index(ascending=False)
honor_counts.sort_values(ascending=True).sort_values(ascending=False).head(30)

## reducing categories

1. misc category
1. aggregating similar categories *seems like a lot of work, let's try if misc is enough * 
    1. mech
        1. worker placement
        1. auction
    1. categ
        1. america wars
    1. honor
        1. count/delete nominations * a LOT of work*
        1. count awards

## creating dummy variables
1. mechanic
1. subdomain
1. family
1. category
1. publisher
1. artist
1. designer


In [ ]:
def drop_one_hot_and_aggregate(df:pd.DataFrame, cutoff_type:str, param:int, agg:bool) -> pd.DataFrame:
    bools = None
    if cutoff_type == "min_freq":
        bools = (df.sum() < param).values
        df_out1 = df[df.columns[~bools]]
    agg_array = df[df.columns[bools]].sum(axis=1) if agg else None
    return df_out1, agg_array

In [ ]:
def aggregate_mlb_clean(df:pd.DataFrame, column_search_str:str, misc:tuple[str,int,bool] = ("min_freq",1000,False)) -> tuple[pd.DataFrame,MultiLabelBinarizer,np.ndarray]:
    '''
    aggregate several columns of the same multi-label variable in one column as lists, one hot encode it and deleted superfluous columns

    df: input dataframe
    misc: Da muss ich mir noch überlegen welche Arten ich will. Top x? x% ? Cutoff wenn zuviele?
    '''
    df[column_search_str] = df.filter(regex= column_search_str + "[0-9]").fillna("NaN").values.tolist()
    mlb = MultiLabelBinarizer()
    mlb_array = mlb.fit_transform(df[column_search_str])
    drop_columns(df, column_search_str)
    df2 = pd.DataFrame(mlb_array, columns= column_search_str + "_" + mlb.classes_, index=df.index).drop(columns=column_search_str + "_" +"NaN")
    df2, agg_array = drop_one_hot_and_aggregate(df2, *misc)
    if agg_array is not None:
        df2[column_search_str + "_misc"] = agg_array
    df_out = df.join(df2)
    return df_out, mlb, mlb_array

In [ ]:
karim2, karim_mlb,karim_mlb_array = aggregate_mlb_clean(bgg_df2.copy(), "category")

In [ ]:
print(len(karim2.columns))
karim2, _ , _ = aggregate_mlb_clean(karim2, "mechanic")
print(len(karim2.columns))
karim2, _ , _ = aggregate_mlb_clean(karim2, "subdomain")
print(len(karim2.columns))

In [ ]:
karim2, _, _ = aggregate_mlb_clean(karim2, "family")
print(len(karim2.columns))


In [ ]:
karim2, _, _ = aggregate_mlb_clean(karim2, "publisher")
print(len(karim2.columns))


## Regression

In [ ]:
from sklearn.preprocessing import MinMaxScaler

def clean_fit_linear_mod(df, response_col, cat_cols, dummy_na, test_size=.3, rand_state=42):
    '''
    INPUT:
    df - a dataframe holding all the variables of interest
    response_col - a string holding the name of the column 
    cat_cols - list of strings that are associated with names of the categorical columns
    dummy_na - Bool holding whether you want to dummy NA vals of categorical columns or not
    test_size - a float between [0,1] about what proportion of data should be in the test dataset
    rand_state - an int that is provided as the random state for splitting the data into training and test 
    
    OUTPUT:
    test_score - float - r2 score on the test data
    train_score - float - r2 score on the test data
    lm_model - model object from sklearn
    X_train, X_test, y_train, y_test - output from sklearn train test split used for optimal model
    '''
    #Drop the rows with missing response values
    df  = df.dropna(subset=[response_col], axis=0)

    #Drop columns with all NaN values
    df = df.dropna(how='all', axis=1)

    #Dummy categorical variables
    #df = create_dummy_df(df, cat_cols, dummy_na)

    # Mean function
    fill_mean = lambda col: col.fillna(col.mean())
    # Fill the mean
    df = df.apply(fill_mean, axis=0)

    #Split into explanatory and response variables
    X = df.drop(response_col, axis=1)
    y = df[response_col]

    #Split into train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=rand_state)

    # krieg das normalisieren (stichwort skaling?) nicht hin 
    #min_max_scaler = MinMaxScaler().fit(X_test)
    #X_norm = min_max_scaler.transform(X) # Keine ahnung was das macht
    #  
    lm_model = LinearRegression() # Instantiate
    lm_model.fit(X_train, y_train) #Fit

    #Predict using your model
    y_test_preds = lm_model.predict(X_test)
    y_train_preds = lm_model.predict(X_train)

    #Score using your model
    test_score = r2_score(y_test, y_test_preds)
    train_score = r2_score(y_train, y_train_preds)

    return test_score, train_score, lm_model, X_train, X_test, y_train, y_test


#Test your function with the above dataset
cat_cols_lst = None
test_score, train_score, lm_model, X_train, X_test, y_train, y_test = clean_fit_linear_mod(karim2.select_dtypes(exclude="object"), 'average', cat_cols_lst, dummy_na=False)
test_score, train_score

In [ ]:
# Das mal ausprobieren und Ergebnisse anschauen https://scikit-learn.org/stable/auto_examples/preprocessing/plot_scaling_importance.html

In [ ]:
#0.3299249671443033, 0.3350201909325803

def coef_weights(coefficients, X_train):
    '''
    INPUT:
    coefficients - the coefficients of the linear model 
    X_train - the training data, so the column names can be used
    OUTPUT:
    coefs_df - a dataframe holding the coefficient, estimate, and abs(estimate)
    
    Provides a dataframe that can be used to understand the most influential coefficients
    in a linear model by providing the coefficient estimates along with the name of the 
    variable attached to the coefficient.
    '''
    coefs_df = pd.DataFrame()
    coefs_df['est_int'] = X_train.columns
    coefs_df['coefs'] = lm_model.coef_
    coefs_df['abs_coefs'] = np.abs(lm_model.coef_)
    coefs_df = coefs_df.sort_values('abs_coefs', ascending=False)
    return coefs_df

#Use the function
coef_df = coef_weights(lm_model.coef_, X_train)

#A quick look at the top results
coef_df

In [ ]:
lm_model.intercept_

# Looking for divisive games

In [ ]:
bgg_df.sort_values(by="usersrated", ascending=False).head(500) #
bgg_df.sort_values(by="stddev", ascending=True)[bgg_df["usersrated"] > 100 ]

#Bins machen für average und mir die mittlere anschauen und stddev nach oben sortieren

# Misc

In [ ]:
bgg_df.iloc[:,:10].hist()

In [ ]:
sns.heatmap(bgg_df.corr().iloc[:10,:10], annot=True, fmt=".2f")

In [ ]:
bgg_df.corr().iloc[:,:10]